# Playing around with interactive maps and my google maps geo data

## Import libraries and data

In [1]:
import pandas as pd
import geoplot as gplt
import matplotlib.pyplot as plt
import descartes
import folium
import geopandas as gpd
from shapely.geometry import Point, Polygon

The crs value refers to the type of projection to be used i.e. how the map of the earth will be projected. We choose the epsg value 4326 as defined below: <br>
" EPSG 4326 defines a full coordinate reference system, providing spatial meaning to otherwise meaningless pairs of numbers. It means "latitude and longitude coordinates on the WGS84 reference ellipsoid." <br> <br>
*source https://gis.stackexchange.com/questions/3334/difference-between-wgs84-and-epsg4326*

In [2]:
crs = {'init':'epsg:4326'}
#epsg = 3857 used by googlemaps

In [3]:
df = pd.read_json('../data/raw/Saved Places.json')

In [4]:
df

,type,features
0,FeatureCollection,"{'geometry': {'coordinates': [-3.7082834, 40.4..."
1,FeatureCollection,"{'geometry': {'coordinates': [8.691251, 50.179..."
2,FeatureCollection,"{'geometry': {'coordinates': [7.3411604, 50.08..."
3,FeatureCollection,"{'geometry': {'coordinates': [-3.703461, 40.41..."
4,FeatureCollection,"{'geometry': {'coordinates': [13.5628678, 46.3..."
...,...,...
915,FeatureCollection,"{'geometry': {'coordinates': [-6.2636588, 53.3..."
916,FeatureCollection,"{'geometry': {'coordinates': [-6.265298, 53.34..."
917,FeatureCollection,"{'geometry': {'coordinates': [-121.9017595, 36..."
918,FeatureCollection,"{'geometry': {'coordinates': [-118.915943, 34...."


## Data processing

The json file from google out of the box is not ready for processing so I cut out fields I am not interested in

In [5]:
df['features'].iloc[0]

{'geometry': {'coordinates': [-3.7082834, 40.4341012], 'type': 'Point'},
 'properties': {'Google Maps URL': 'http://maps.google.com/?cid=12696808076577241862',
  'Location': {'Address': 'Calle de Vallehermoso, 36, 28015 Madrid, Spain',
   'Business Name': 'Mercado de Vallehermoso',
   'Country Code': 'ES',
   'Geo Coordinates': {'Latitude': '40.4341012', 'Longitude': '-3.7082834'}},
  'Published': '2020-05-21T17:10:13Z',
  'Title': 'Mercado de Vallehermoso',
  'Updated': '2020-05-21T17:10:13Z'},
 'type': 'Feature'}

In [6]:
def process_gmaps_json(data, country_id):
    lat = []
    long = []
    name = []
    for item in data['features']:
        coords = item["geometry"]
        country = item['properties']
        country_2 = country['Location']
        if 'Country Code' in country_2.keys():
            country3=country_2['Country Code']
            if coords["type"] == "Point" and country3==country_id:
                lat.append(coords['coordinates'][0])
                long.append(coords['coordinates'][1])
                name.append(country['Title'])
    d_tuples = list(zip(lat,long,name))
    df = pd.DataFrame(d_tuples,columns=['longitude','latitude','name'])
    return df

Let's use Iceland as an example i.e. use country_id = 'IS'. The function will return the latitude, longitude and places I visited in Iceland.

In [7]:
df = process_gmaps_json(df,'IS')

In [8]:
df

,longitude,latitude,name
0,-19.370416,63.530431,Sólheimajökull glacier parking lot
1,-20.309351,64.137413,Secret Lagoon Hot Spring
2,-20.993864,63.936971,Selfoss HI Hostel
3,-21.129550,64.255920,Thingvellir National Park
4,-21.118298,64.265920,Öxarárfoss
5,-21.938231,64.147522,Eymundsson
6,-21.923427,64.144499,Svarta Kaffið
7,-16.918065,65.649192,Hlid Hostel
8,-16.847968,65.630899,Mývatn Nature Baths
9,-16.809311,65.640914,Hverir


## Geodata

We now need to create a geodataframe using the geoplot library. The geodataframe includes a geometry field (here a point) that will be used to locate items on the map.

In [15]:
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

In [16]:
geo_df = gpd.GeoDataFrame(df, crs=crs,geometry=geometry)

In [17]:
geo_df

,longitude,latitude,name,geometry
0,-19.370416,63.530431,Sólheimajökull glacier parking lot,POINT (-19.37042 63.53043)
1,-20.309351,64.137413,Secret Lagoon Hot Spring,POINT (-20.30935 64.13741)
2,-20.993864,63.936971,Selfoss HI Hostel,POINT (-20.99386 63.93697)
3,-21.129550,64.255920,Thingvellir National Park,POINT (-21.12955 64.25592)
4,-21.118298,64.265920,Öxarárfoss,POINT (-21.11830 64.26592)
5,-21.938231,64.147522,Eymundsson,POINT (-21.93823 64.14752)
6,-21.923427,64.144499,Svarta Kaffið,POINT (-21.92343 64.14450)
7,-16.918065,65.649192,Hlid Hostel,POINT (-16.91807 65.64919)
8,-16.847968,65.630899,Mývatn Nature Baths,POINT (-16.84797 65.63090)
9,-16.809311,65.640914,Hverir,POINT (-16.80931 65.64091)


## Display interactive maps with Folium

Below is an example of how to use folium using specific longitude and latitudes. Here we will show the Eiffel tower.

In [41]:
eiffel_tower = folium.Map(location=[48.8583736,2.2922926],zoom_start=15)

In [42]:
display(eiffel_tower)

Let's now display Iceland using my geo data. We need to define the center of the map and the level of zoom (where zoom_start=1 shows the whole world). We will use a spot in the capital as the center of the map.

In [57]:
center_of_the_map = geo_df.geometry[28]
map_location = [iceland_center.y, iceland_center.x]

In [55]:
int_map = folium.Map(location=map_location,zoom_start=6)

In [56]:
display(int_map)

We can now fully exploit the geodata from google maps and even add markers in places I visited as well as labels to know exactly which place these markers refer to.

In [58]:
for row in geo_df.iterrows():
    row_val = row[1]
    location = [row_val['latitude'],row_val['longitude']]
    popup = popup = '<strong>' + row_val['name'] +'</strong>'
    marker = folium.Marker(location=location,popup=popup)
    marker.add_to(int_map)

In [59]:
display(int_map)